In [ ]:
pip uninstall transformers


In [ ]:
pip install transformers

In [21]:
import tensorflow as tf
from tensorflow import keras
import os
import string
import chromadb
os.environ["KAGGLE_USERNAME"] = "*****"
os.environ["KAGGLE_KEY"] = "****"
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline, Conversation


ImportError: cannot import name 'Conversation' from 'transformers' (c:\Users\raymo\anaconda3\Lib\site-packages\transformers\__init__.py)

In [20]:
print(Conversation)

NameError: name 'Conversation' is not defined

In [7]:
documents = []
metadata = []
ids = []

with open('data/spells.txt', 'r', encoding='utf-8') as spells:
    for line in spells:
        line=line.split(" - ") #spells are separated by their description by a " - "
        metadata.append({'spell': line[0]}) #use the spell name as metadata. ChromaDB requires a dictionary here
        documents.append(line[1]) #use the description as documents
        ids.append(line[0]) #we also need an ID so use the spell name again.
print(ids)

['Aberto', 'Accio', 'Aguamenti', 'Alohomora', 'Anapneo', 'Aparecium', 'Apparate', 'Ascendio', 'Avada Kedavra', 'Avis', 'Bat-Bogey Hex', 'Bombardo', 'Brackium Emendo', 'Capacious Extremis', 'Confundo', 'Conjunctivitis Curse', 'Crinus Muto', 'Crucio', 'Diffindo', 'Disillusionment Charm', 'Disapparate', 'Engorgio', 'Episkey', 'Expecto patronum', 'Erecto', 'Evanesco', 'Expelliarmus', 'Ferula', 'Fidelius Charm', 'Fiendfyre Curse', 'Finite Incantatem', 'Furnunculus Curse', 'Geminio', 'Glisseo', 'Homenum Revelio', 'Homonculus Charm', 'Immobulus', 'Impedimenta', 'Incarcerous', '*Imperio', 'Impervius', 'Incendio', 'Langlock', 'Legilimens', 'Levicorpus', 'Locomotor Mortis', 'Lumos', 'Morsmordre', 'Mucus Ad Nauseam', 'Muffliato', 'Nox', 'Obliviate', 'Obscuro', 'Oculus Reparo', 'Oppugno', 'Petrificus Totalus', 'Periculum', 'Piertotum Locomotor', 'Protean Charm', 'Protego', 'Reducto', 'Reducio', 'Renneverate', 'Reparifors', 'Reparo', 'Rictusempra', 'Riddikulus', 'Scourgify', '*Sectumsempra', 'Serpe

In [8]:

client = chromadb.Client()

# create or open collection named 'spells' 
spells_collection = client.get_or_create_collection("spells")

# add spells to the collection, ignored for ids that already exist 
spells_collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

C:\Users\raymo\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 12.8MiB/s]


In [9]:
user_input="How do I create light"
results = spells_collection.query(
    query_texts=[user_input],
    n_results=1
)
results

{'ids': [['Lumos']],
 'distances': [[1.327547311782837]],
 'metadatas': [[{'spell': 'Lumos'}]],
 'embeddings': None,
 'documents': [["Illuminates the caster's wand\n"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [11]:
result=results['metadatas'][0][0]['spell']+" - " + results['documents'][0][0][:-1]
result

"Lumos - Illuminates the caster's wand"

In [13]:


# load_in_8bit: lower precision but saves a lot of GPU memory
# device_map=auto: loads the model across multiple GPUs
chatbot = pipeline("conversational", model="BramVanroy/GEITje-7B-ultra", model_kwargs={"load_in_8bit": True}, device_map="auto")

start_messages = [
    {"role": "system", "content": "Je bent een grappige chatbot die Bert heet. Je maakt vaak mopjes."},
    {"role": "user", "content": "Hallo, ik ben Bram. Ik wil vanavond graag een film kijken. Heb je enkele suggesties?"}
]
conversation = Conversation(start_messages)
conversation = chatbot(conversation)
response = conversation.messages[-1]["content"]
print(response)

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 096635dc-bb53-4b36-80b5-46c53461be82)')

In [ ]:
prompt = """
Recommended spell: {result}
User input: {user_input} 
Suggest a spell based on the recommended spell and the user input."""

prompt=prompt.format(user_input=user_input, result=result)
print ("prompt: ", prompt)
with tf.device('CPU: 0'): #Force on CPU for both apple silicon and low GPU mem configurations
        inference = model.generate(prompt, max_length = 100)
print(inference)